In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('Listings.csv',encoding = 'unicode_escape')

/tmp/ipython-input-3260820350.py:1: DtypeWarning: Columns (5,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Listings.csv',encoding = 'unicode_escape')


Percentage of missing value for each column

In [28]:
df.count()

,0
listing_id,279712
name,279537
host_id,279712
host_since,279547
host_location,278872
host_response_time,150930
host_response_rate,150930
host_acceptance_rate,166625
host_is_superhost,279547
host_total_listings_count,279547


In [12]:
round((df.isnull().sum()/df.count()),5) * 100

,0
listing_id,0.000
name,0.063
host_id,0.000
host_since,0.059
host_location,0.301
host_response_time,85.326
host_response_rate,85.326
host_acceptance_rate,67.869
host_is_superhost,0.059
host_total_listings_count,0.059


Dropping rows with missing values in columns who have really small no. of missing values.

In [13]:
clean_df = df.dropna(subset = ['name','host_since','host_location','host_is_superhost','host_total_listings_count','host_has_profile_pic','host_identity_verified'])

In [15]:
clean_df['host_response_time'].describe()

,host_response_time
count,150652
unique,4
top,within an hour
freq,83350


In [18]:
clean_df['host_response_time'].isnull().sum()

np.int64(128046)

In [35]:
clean_df['host_response_time'].value_counts()

,count
host_response_time,
within an hour,154131
within a few hours,53471
within a day,43137
a few days or more,27959


In [34]:
import pandas as pd
import numpy as np

# Your observed distribution
freqs = {
    "within an hour": 83352,
    "within a few hours": 28844,
    "within a day": 23365,
    "a few days or more": 15099
}

# Convert to probabilities
total = sum(freqs.values())
probs = {k: v/total for k, v in freqs.items()}

# Create categories and their probabilities
categories = list(probs.keys())
prob_values = list(probs.values())

# Mask for missing values
mask = clean_df['host_response_time'].isna()

# Number of missing values
n_missing = mask.sum()

# Randomly sample categories based on global proportions
imputed_values = np.random.choice(categories, size=n_missing, p=prob_values)

# Fill missing values
clean_df.loc[mask, 'host_response_time'] = imputed_values


In [40]:
clean_df['host_response_rate'].skew()

np.float64(-2.2708184933680013)

In [41]:
clean_df.loc[clean_df['host_response_rate'].isna(), 'host_response_rate'] = clean_df['host_response_rate'].median()

In [43]:
clean_df['host_acceptance_rate'].skew()

np.float64(-1.8644162821051808)

In [44]:
clean_df.loc[clean_df['host_acceptance_rate'].isna(), 'host_acceptance_rate'] = clean_df['host_acceptance_rate'].median()

In [46]:
df['district'].isnull().sum()


np.int64(242700)

In [47]:
df['district'].describe()

,district
count,37012
unique,5
top,Manhattan
freq,16553


In [48]:
df['district'].value_counts()

,count
district,
Manhattan,16553
Brooklyn,14474
Queens,4704
Bronx,992
Staten Island,289


In [54]:
check = clean_df.groupby('neighbourhood')['district'].nunique()
print(check.value_counts())


district
0    440
1    220
Name: count, dtype: int64


In [55]:
# Step 1: Build mapping from neighbourhood → district (unique ones only)
mapping = (
    clean_df.dropna(subset=['district'])
      .groupby('neighbourhood')['district']
      .agg(lambda x: x.mode().iloc[0])   # ensures one-to-one mapping
)

# Step 2: Fill missing districts using mapping
mask = clean_df['district'].isna()
clean_df.loc[mask, 'district'] = clean_df.loc[mask, 'neighbourhood'].map(mapping)

# Step 3: Fallback - fill still-missing values with global mode of district
global_mode = clean_df['district'].mode().iloc[0]
clean_df['district'].fillna(global_mode, inplace=True)

# Step 4: Check results
print("Global mode used for fallback:", global_mode)
print("Missing after imputation:", clean_df['district'].isna().sum())
print(clean_df['district'].value_counts().head(10))  # preview top districts


Global mode used for fallback: Manhattan
Missing after imputation: 0
district
Manhattan        258303
Brooklyn          14431
Queens             4689
Bronx               987
Staten Island       288
Name: count, dtype: int64


/tmp/ipython-input-1479090659.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  clean_df['district'].fillna(global_mode, inplace=True)
/tmp/ipython-input-1479090659.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['district'].fillna(global_mode, inplace=True)


In [57]:
clean_df['bedrooms'].describe()

,bedrooms
count,249388.000000
mean,1.515406
std,1.153140
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,50.000000


In [62]:
clean_df['bedrooms'].skew()

np.float64(13.695632141437077)

In [61]:
clean_df.loc[clean_df['bedrooms'].isna(), 'bedrooms'] = round(clean_df['bedrooms'].median(),0)

In [64]:
clean_df['review_scores_rating'].describe()

,review_scores_rating
count,187851.000000
mean,93.406530
std,10.062766
min,20.000000
25%,91.000000
50%,96.000000
75%,100.000000
max,100.000000


In [65]:
clean_df['review_scores_rating'].skew()

np.float64(-3.758240069504656)

In [67]:
clean_df.loc[clean_df['review_scores_rating'].isna(), 'review_scores_rating'] = round(clean_df['review_scores_rating'].median(),0)

In [68]:
clean_df['review_scores_accuracy'].skew()

np.float64(-4.214235842815134)

In [69]:
clean_df.loc[clean_df['review_scores_accuracy'].isna(), 'review_scores_accuracy'] = round(clean_df['review_scores_accuracy'].median(),0)

In [70]:
clean_df['review_scores_cleanliness'].skew()

np.float64(-3.0273956492814817)

In [72]:
clean_df['review_scores_cleanliness'].sample(5)

,review_scores_cleanliness
123574,NaN
208786,NaN
6316,9.0
80740,NaN
180912,NaN


In [73]:
clean_df.loc[clean_df['review_scores_cleanliness'].isna(), 'review_scores_cleanliness'] = round(clean_df['review_scores_cleanliness'].median(),0)

In [75]:
clean_df['review_scores_checkin'].skew()

np.float64(-5.262259297558439)

In [76]:
clean_df['review_scores_checkin'].sample(5)

,review_scores_checkin
270858,10.0
243507,10.0
194083,NaN
189777,9.0
95489,10.0


In [77]:
clean_df.loc[clean_df['review_scores_checkin'].isna(), 'review_scores_checkin'] = round(clean_df['review_scores_checkin'].median(),0)

In [78]:
clean_df['review_scores_communication'].skew()

np.float64(-5.229436382842495)

In [79]:
clean_df['review_scores_checkin'].sample(5)

,review_scores_checkin
105725,10.0
67801,10.0
106639,10.0
81510,10.0
250380,8.0


In [80]:
clean_df.loc[clean_df['review_scores_communication'].isna(), 'review_scores_communication'] = round(clean_df['review_scores_communication'].median(),0)

In [86]:
round((clean_df.isnull().sum()/clean_df.count()),5) * 100

,0
listing_id,0.0
name,0.0
host_id,0.0
host_since,0.0
host_location,0.0
host_response_time,0.0
host_response_rate,0.0
host_acceptance_rate,0.0
host_is_superhost,0.0
host_total_listings_count,0.0


In [82]:
clean_df['review_scores_location'].skew()

np.float64(-4.315082980549189)

In [83]:
clean_df.loc[clean_df['review_scores_location'].isna(), 'review_scores_location'] = round(clean_df['review_scores_location'].median(),0)

In [84]:
clean_df['review_scores_value'].skew()

np.float64(-3.2519682422810927)

In [85]:
clean_df.loc[clean_df['review_scores_value'].isna(), 'review_scores_value'] = round(clean_df['review_scores_value'].median(),0)

In [87]:
df.to_csv('listings_cleaned.csv')